# Weighted Least Squares

In [ ]:
from gnss_lib_py.parsers.android import AndroidDerived2021
from gnss_lib_py.algorithms.snapshot import solve_wls
from gnss_lib_py.utils.visualizations import plot_map

In [ ]:
# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

Solve for the Weighted Least Squares position estimate simply by passing the measurement data.

In [ ]:
state_wls = solve_wls(derived_data)

Plot the ECEF x and ECEF y computed position estimate of the receiver

In [ ]:
plot_map(state_wls)

# Extended Kalman Filter

In [ ]:
from gnss_lib_py.parsers.android import AndroidDerived2021
from gnss_lib_py.algorithms.gnss_filters import solve_gnss_ekf
from gnss_lib_py.utils.visualizations import plot_map

In [ ]:
# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

Solve for the Weighted Least Squares position estimate simply by passing the measurement data.

In [ ]:
state_ekf = solve_gnss_ekf(derived_data)

Plot the ECEF x and ECEF y computed position estimate of the receiver

In [ ]:
plot_map(state_ekf)

# Residuals

In [ ]:
from gnss_lib_py.algorithms.residuals import solve_residuals
from gnss_lib_py.utils.visualizations import plot_metric_by_constellation
galileo_data = derived_data.where("gnss_id","galileo")

Solve for residuals using the estimated state. A new "residuals_m" row is added to derived_data

In [ ]:
solve_residuals(galileo_data, state_wls, inplace=True)

Plot the residuals using the `plot_residuals()` function. The residuals are broken up constellation and signal type and plotted on separate figures.

In [ ]:
figs = plot_metric_by_constellation(galileo_data, "gps_millis", "residuals_m")